# Simplification API

Ideal API of the simplification, all wrapped to a single function.

In [1]:
import warnings

import folium
import geopandas
import sgeop

Filter out the RuntimeWarning showing on Apple Silicon.

In [2]:
warnings.filterwarnings(
    "ignore",
    category=RuntimeWarning,
    message="invalid value encountered in intersection",
)

Read road data

In [3]:
roads = geopandas.read_parquet("/Users/martin/Downloads/streets_69300.parquet")

In [17]:
roads = sgeop.fix_topology(roads)

In [13]:
buildings = geopandas.read_parquet(
    "/Users/martin/Downloads/buildings_chars_69300.parquet", columns=["geometry"]
)

Simplify network

In [18]:
new_roads = sgeop.simplify_network(roads, exclusion_mask=buildings.geometry)

/Users/martin/Git/simplification/core/geometry.py:196: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/Users/martin/Git/simplification/core/algorithms/artifacts.py:267: UserWarning: Could not create a connection as it would lead outside of the artifact.
  adds, splitters = snap_to_targets(
/Users/martin/Git/simplification/core/geometry.py:196: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/Users/martin/Git/simplification/core/algorithms/simplify.py:337: UserWarning: An error occured at location POINT (4634734.833449237 3007874.1868347814). The artifact has not been simplified. The original message:
can only convert an array of size 1 to a Python scalar
  roads_cleaned = simplify_singletons(


In [19]:
new_roads.to_parquet("streets_69300_simplified.parquet")

Visualise

In [ ]:
new_roads = geopandas.read_parquet("streets_69300_simplified.parquet")

In [ ]:
m = roads.explore(
    max_zoom=52,
    tiles="cartodb positron",
    color="black",
    prefer_canvas=True,
    highlight_kwds=dict(color="red"),
)
m = new_roads.explore("_status", m=m, highlight_kwds=dict(color="red"))
folium.LayerControl().add_to(m)
m